# 1. import data & libraries

In [259]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from __future__ import division
from itertools import chain
from nltk.probability import *
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
import re
import time

In [17]:
# load train and test data
train = pandas.read_csv("/Users/yezhenwang/Documents/Monash/Monash semester 2 2018/Quora/data/train.csv")
test = pandas.read_csv("/Users/yezhenwang/Documents/Monash/Monash semester 2 2018/Quora/data/test.csv")

In [9]:
ls

README.md          Tokenize.ipynb     all/               quora_demo1.ipynb


In [12]:
train = pd.read_csv("all/train.csv")
test = pd.read_csv("all/test.csv")

# 2. Randomly pick some data to chek

In [222]:
# Randomly pick rows from dataset
# target = -1, don't care about it
# target = 1, insincere
# target = 0, normal label
# default display count is 10.
def pickRawDataByTarget(target, num=10):
    random_list = []
    if target == -1:
        for i in range(num):
            random_list.append(np.random.random_integers(len(train)+1))
        for row in random_list:
            print("Q:%d \n%s || Target: %s"%(row, train.iloc[row,1], train.iloc[row,2]))
    else:
        # Create a correspond target index list at first
        targetList = (train[train['target'] == target].index).tolist()
        for i in range(num):
            random_list.append(np.random.random_integers(len(targetList)))
        for index in random_list:
            print("Q:%d \n%s || Target: %s"%(targetList[index], train.iloc[targetList[index],1], train.iloc[targetList[index],2]))

In [221]:
pickRawDataByTarget(-1)

Q:1104471 
Do you have that one skill or a set of few skills that you know will never desert you when you need it? How did you develop it? And how did you discover that those are the skills you wanted to develop? || Target: 0
Q:203810 
If a student worker on the F1 visa program works for 21 hours in a certain week just once by mistake I that a very serious offence? What imapct will it have on other H1B application in th future || Target: 0
Q:1085416 
Should I let my son watch porn and masturbate with his twin sister? || Target: 1
Q:861616 
What do you, as an atheist, think about Tim O'Neill and atheists who believe in the existence of Jesus and deny the history of Christian atrocities? || Target: 0
Q:420564 
What are the most effective techniques and procedures to perform a tubal ligation surgery? || Target: 0
Q:166543 
Whatre some of the best books to read for Java, application and system design? || Target: 0
Q:1088805 
When will Hindus rise up and invade the world? || Target: 1
Q:105

/Users/solar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. Please call randint(1, 1306123 + 1) instead
  if __name__ == '__main__':


In [200]:
#train_token_tag_df.iloc[819941,2]

## Lines might need caution
1. No.338434 12th -> twelveth<br>
2. No.1286381 Military Service: Capital case in the middle of sentence, check it whether could be bigram.
3. No.968909 man-bun with "-"
4. No.421977 22-Jun-2018, 'JJ' POS (I think it's Wrong?)
5. No.819941 Rice production/distribution/processing, dealing with '/'
    <br>Q: Stemming & N-gram order? (ref: https://stackoverflow.com/questions/47219389/compute-word-n-grams-on-original-text-or-after-lemma-stemming-process)
6. No.1298566 mis-spelling.
7. No.713198 ...Math question.


# 3. Related Functions for pre-processing: case_Normaliztion, spec_Replacement, etc.

In [77]:
def case_Normalize(line):
    '''
        'Originially used in my FIT5196 assignment, manully picked some end-tokens of a sentence'
        Write a regex to match all words with a first capital letter in front of line or with some marks 
        ahead except the marks which would occur within a sentecne like [)(&,;-]. I think it not performed perfect but 
        it is better than sent_detector.tokenize()
        
        'Here is a caution in 5196, I'm not sure whether it happened here, but it seems no side-effect'
        Caution: as data always contain many space tokens, I replace them with single space at first.
    '''
    casePattern = re.compile(r"(^[A-Z]\w*|[^\w\&,\)\(;-]\s[A-Z][a-z]+)")
    # Some files have too many space tokens among two words in a sentecne
    line = re.sub( '\s+', ' ', line).strip()
    # string.lower() used to normalise the capital letter.
    return casePattern.sub(lambda x: x.group(0).lower(), line)

In [158]:
def spec_Replacement(line):
    '''
        A regex for split specific punctuation like '/'
        1. replace '/' with ' or ' for furhter tokenize.
        2. replace '%' with ' percentage'
        3. replace '$' with 'dollar '
    '''
    line = re.sub( '\/', ' or ', line).strip()
    line = re.sub( '\$', 'dollars ', line).strip()
    line = re.sub( '\%', ' percentage', line).strip()
    

    return line

In [224]:
# Add any other functions here if need.
# Add more functions inside #3 part. Then add them into pre-processing function
def dealing_Non_Alphanumeric(line):
    pass

In [223]:
# This cell used for simple test
line1 = train.iloc[819941,1]
line1 = re.sub( '\s+', ' ', line1).strip()
#line1
line1 = case_Normalize(line1)
spec_Replacement(line1)

'how can I invest in Rice production or distribution or processing in Nigeria?'

# 4. Pre-processing

In [160]:
# Add related pre-processing function here
def pre_process(line):
    line = case_Normalize(line)
    line = spec_Replacement(line)
    
    return line

In [230]:
# Define funtion to get a stopword list
# Default using nltk corpus.
# Might add alterations.
def get_StopWords():
    return set(stopwords.words('english'))

In [81]:
# build question lists for train and test
train_question_list = train["question_text"].tolist()
test_question_list = test["question_text"].tolist()

## Q: Stemming or Lemmatization

> In FIT5196, we are required to do stemming.<br>
> https://stackoverflow.com/questions/49354665/should-i-perform-both-lemmatization-and-stemming

In [253]:
len(test_question_list)

56370

In [263]:
# tokenize questions in train
t0 = time.clock()
train_token_list = []
for i in range(len(train_question_list)):
    line = pre_process(train_question_list[i])
    unigram_tokens = nltk.word_tokenize(line)
    
    stemmer = PorterStemmer()
    stemed_tokens = list(map(lambda x: stemmer.stem(x) if not x[0].isupper() else x, unigram_tokens))
    stopped_tokens = [w for w in stemed_tokens if w not in get_StopWords()]
    
    # final_tokens remove any tokens have a tiny length (< 3)
    final_tokens = list(filter(lambda x: len(x) >= 3, stopped_tokens))
    train_token_list.append(final_tokens)

    if i%4000 == 0:
        t1 = time.clock()
        print("Processing {0}, {1} percent of total 1306122, time cost {2}".format(i, i/1306122, round((t1-t0), 2)))
        t0 = t1

Processing 0, 0.0 percent of total 1306122, time cost 0.01
Processing 4000, 0.0030625010527347367 percent of total 1306122, time cost 9.99
Processing 8000, 0.0061250021054694735 percent of total 1306122, time cost 10.37
Processing 12000, 0.00918750315820421 percent of total 1306122, time cost 9.79
Processing 16000, 0.012250004210938947 percent of total 1306122, time cost 9.85
Processing 20000, 0.015312505263673684 percent of total 1306122, time cost 9.71
Processing 24000, 0.01837500631640842 percent of total 1306122, time cost 9.79
Processing 28000, 0.02143750736914316 percent of total 1306122, time cost 9.73
Processing 32000, 0.024500008421877894 percent of total 1306122, time cost 9.86
Processing 36000, 0.027562509474612633 percent of total 1306122, time cost 9.68
Processing 40000, 0.03062501052734737 percent of total 1306122, time cost 9.89
Processing 44000, 0.033687511580082104 percent of total 1306122, time cost 9.74
Processing 48000, 0.03675001263281684 percent of total 1306122, 

Processing 416000, 0.31850010948441265 percent of total 1306122, time cost 9.62
Processing 420000, 0.32156261053714735 percent of total 1306122, time cost 9.75
Processing 424000, 0.3246251115898821 percent of total 1306122, time cost 9.67
Processing 428000, 0.32768761264261687 percent of total 1306122, time cost 9.8
Processing 432000, 0.33075011369535157 percent of total 1306122, time cost 9.83
Processing 436000, 0.3338126147480863 percent of total 1306122, time cost 9.76
Processing 440000, 0.3368751158008211 percent of total 1306122, time cost 9.69
Processing 444000, 0.3399376168535558 percent of total 1306122, time cost 9.64
Processing 448000, 0.34300011790629054 percent of total 1306122, time cost 9.65
Processing 452000, 0.3460626189590253 percent of total 1306122, time cost 9.71
Processing 456000, 0.34912512001176 percent of total 1306122, time cost 9.58
Processing 460000, 0.35218762106449475 percent of total 1306122, time cost 9.63
Processing 464000, 0.3552501221172295 percent of 

Processing 832000, 0.6370002189688253 percent of total 1306122, time cost 9.8
Processing 836000, 0.6400627200215601 percent of total 1306122, time cost 9.77
Processing 840000, 0.6431252210742947 percent of total 1306122, time cost 9.72
Processing 844000, 0.6461877221270295 percent of total 1306122, time cost 9.83
Processing 848000, 0.6492502231797642 percent of total 1306122, time cost 9.73
Processing 852000, 0.652312724232499 percent of total 1306122, time cost 9.78
Processing 856000, 0.6553752252852337 percent of total 1306122, time cost 9.84
Processing 860000, 0.6584377263379684 percent of total 1306122, time cost 9.66
Processing 864000, 0.6615002273907031 percent of total 1306122, time cost 9.79
Processing 868000, 0.6645627284434379 percent of total 1306122, time cost 9.73
Processing 872000, 0.6676252294961726 percent of total 1306122, time cost 9.73
Processing 876000, 0.6706877305489074 percent of total 1306122, time cost 9.85
Processing 880000, 0.6737502316016422 percent of total

Processing 1248000, 0.955500328453238 percent of total 1306122, time cost 9.85
Processing 1252000, 0.9585628295059726 percent of total 1306122, time cost 9.6
Processing 1256000, 0.9616253305587074 percent of total 1306122, time cost 9.76
Processing 1260000, 0.9646878316114421 percent of total 1306122, time cost 9.63
Processing 1264000, 0.9677503326641769 percent of total 1306122, time cost 9.93
Processing 1268000, 0.9708128337169116 percent of total 1306122, time cost 9.77
Processing 1272000, 0.9738753347696463 percent of total 1306122, time cost 9.81
Processing 1276000, 0.976937835822381 percent of total 1306122, time cost 9.77
Processing 1280000, 0.9800003368751158 percent of total 1306122, time cost 9.77
Processing 1284000, 0.9830628379278505 percent of total 1306122, time cost 9.78
Processing 1288000, 0.9861253389805853 percent of total 1306122, time cost 9.67
Processing 1292000, 0.98918784003332 percent of total 1306122, time cost 9.72
Processing 1296000, 0.9922503410860547 percen

In [256]:
# tokenize questions in test
test_token_list = []
for i in range(len(test_question_list)):
    line = pre_process(test_question_list[i])
    unigram_tokens = nltk.word_tokenize(line)

    stemmer = PorterStemmer()
    stemed_tokens = list(map(lambda x: stemmer.stem(x) if not x[0].isupper() else x, unigram_tokens))
    stopped_tokens = [w for w in stemed_tokens if w not in get_StopWords()]
    
    # final_tokens remove any tokens have a tiny length (< 3)
    final_tokens = list(filter(lambda x: len(x) >= 3, stopped_tokens))
    test_token_list.append(final_tokens)
    
    if i%200 == 0:
        print("Processing {0}, {1} percent of total 56370".format(i, i/56370))

Processing 0, 0.0 percent of total 56370
Processing 200, 0.003547986517651233 percent of total 56370
Processing 400, 0.007095973035302466 percent of total 56370
Processing 600, 0.010643959552953698 percent of total 56370
Processing 800, 0.014191946070604932 percent of total 56370
Processing 1000, 0.017739932588256166 percent of total 56370
Processing 1200, 0.021287919105907396 percent of total 56370
Processing 1400, 0.02483590562355863 percent of total 56370
Processing 1600, 0.028383892141209864 percent of total 56370
Processing 1800, 0.031931878658861094 percent of total 56370
Processing 2000, 0.03547986517651233 percent of total 56370
Processing 2200, 0.03902785169416356 percent of total 56370
Processing 2400, 0.04257583821181479 percent of total 56370
Processing 2600, 0.04612382472946603 percent of total 56370
Processing 2800, 0.04967181124711726 percent of total 56370
Processing 3000, 0.05321979776476849 percent of total 56370
Processing 3200, 0.05676778428241973 percent of total 5

Processing 27400, 0.4860741529182189 percent of total 56370
Processing 27600, 0.48962213943587013 percent of total 56370
Processing 27800, 0.4931701259535214 percent of total 56370
Processing 28000, 0.4967181124711726 percent of total 56370
Processing 28200, 0.5002660989888238 percent of total 56370
Processing 28400, 0.5038140855064751 percent of total 56370
Processing 28600, 0.5073620720241263 percent of total 56370
Processing 28800, 0.5109100585417775 percent of total 56370
Processing 29000, 0.5144580450594288 percent of total 56370
Processing 29200, 0.51800603157708 percent of total 56370
Processing 29400, 0.5215540180947312 percent of total 56370
Processing 29600, 0.5251020046123824 percent of total 56370
Processing 29800, 0.5286499911300337 percent of total 56370
Processing 30000, 0.532197977647685 percent of total 56370
Processing 30200, 0.5357459641653362 percent of total 56370
Processing 30400, 0.5392939506829874 percent of total 56370
Processing 30600, 0.5428419372006387 perce

Processing 54800, 0.9721483058364379 percent of total 56370
Processing 55000, 0.9756962923540891 percent of total 56370
Processing 55200, 0.9792442788717403 percent of total 56370
Processing 55400, 0.9827922653893916 percent of total 56370
Processing 55600, 0.9863402519070428 percent of total 56370
Processing 55800, 0.989888238424694 percent of total 56370
Processing 56000, 0.9934362249423452 percent of total 56370
Processing 56200, 0.9969842114599965 percent of total 56370


In [264]:
#test_token_list

In [265]:
# tag train_token_list
train_token_tag = []
for i in train_token_list:
    train_token_tag.append(nltk.pos_tag(i))

test_token_tag = []
# tag test_token_list
for i in test_token_list:
    test_token_tag.append(nltk.pos_tag(i))

In [266]:
# finalize train and test tokenization
train_token_tag_df = pd.DataFrame({"question_text": train_question_list,
                                  "tokened_question": train_token_list,
                                  "tagged_question": train_token_tag})

test_token_tag_df = pd.DataFrame({"question_text": test_question_list,
                                  "tokened_question": test_token_list,
                                  "tagged_question": test_token_tag})

In [267]:
#train_token_tag_df.iloc[103501,2]

In [268]:
train_token_tag_df.head()

,question_text,tokened_question,tagged_question
0,How did Quebec nationalists see their province...,"[Quebec, nationalist, see, provinc, nation, 1960]","[(Quebec, NNP), (nationalist, NN), (see, VBP),..."
1,"Do you have an adopted dog, how would you enco...","[adopt, dog, would, encourag, peopl, adopt, shop]","[(adopt, JJ), (dog, NN), (would, MD), (encoura..."
2,Why does velocity affect time? Does velocity a...,"[whi, doe, veloc, affect, time, doe, veloc, af...","[(whi, NN), (doe, JJ), (veloc, NN), (affect, J..."
3,How did Otto von Guericke used the Magdeburg h...,"[Otto, von, Guericke, use, Magdeburg, hemispher]","[(Otto, NNP), (von, NNP), (Guericke, NNP), (us..."
4,Can I convert montra helicon D to a mountain b...,"[convert, montra, helicon, mountain, bike, cha...","[(convert, NN), (montra, JJ), (helicon, NN), (..."


In [269]:
test_token_tag_df.head()

,question_text,tokened_question,tagged_question
0,My voice range is A2-C5. My chest voice goes u...,"[voic, rang, A2-C5, chest, voic, goe, includ, ...","[(voic, NN), (rang, VBD), (A2-C5, NNP), (chest..."
1,How much does a tutor earn in Bangalore?,"[much, doe, tutor, earn, Bangalore]","[(much, JJ), (doe, NN), (tutor, NN), (earn, VB..."
2,What are the best made pocket knives under $20...,"[best, made, pocket, knive, dollar, 200-300]","[(best, JJS), (made, VBN), (pocket, NN), (kniv..."
3,Why would they add a hypothetical scenario tha...,"[whi, would, add, hypothet, scenario, imposs, ...","[(whi, NN), (would, MD), (add, VB), (hypothet,..."
4,What is the dresscode for Techmahindra freshers?,"[dresscod, Techmahindra, fresher]","[(dresscod, NN), (Techmahindra, NNP), (fresher..."


In [270]:
#test_token_tag_df.tagged_question[2]

In [271]:
#test_token_tag_df.question_text[2]